In [107]:
import csv
import json
from openai import OpenAI

In [24]:


client = OpenAI()

def generate_product_reviews(num_reviews):
    reviews = []
    prompt_text = f"Napíš {num_reviews} pozitívnych a negatívnych recenzií"
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant that generates unique reviews of products in Slovak with maximum of 25 words and different lengths into csv with columns review_text and sentiment (1-positive, 0-negative)"},
            {"role": "user", "content": prompt_text}
        ],
    )

    text = completion.choices[0].message.content 
    reviews.append(text)
    return reviews


data = []
data.extend(generate_product_reviews(5)) 
# data.extend(generate_product_reviews(5))  

# print(data)
# with open('product_sentiment_analysis_data.csv', 'w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(["review_text", "sentiment"]) 
#     writer.writerows(data)


['review_text,sentiment\n"Fantastický výrobok! Vysoko odporúčam.",1\n"Veľmi efektívny, splnil moje očakávania.",1\n"Skvelá kvalita, rýchle doručenie.",1\n"Žiadne negatíva, dokonalý produkt.",1\n"Nesplnil moje očakávania, sklamanie.",0\n"Horšia kvalita ako som čakal.",0\n"Zlá hodnota za peniaze, neodporúčam.",0\n"Doručenie oneskorené, sklamanie.",0\n"Nedostatočná efektivita, neodporúčam.",0\n"Menej kvalitný ako vyzerá, sklamanie.",0']


In [25]:
print(data)

['review_text,sentiment\n"Fantastický výrobok! Vysoko odporúčam.",1\n"Veľmi efektívny, splnil moje očakávania.",1\n"Skvelá kvalita, rýchle doručenie.",1\n"Žiadne negatíva, dokonalý produkt.",1\n"Nesplnil moje očakávania, sklamanie.",0\n"Horšia kvalita ako som čakal.",0\n"Zlá hodnota za peniaze, neodporúčam.",0\n"Doručenie oneskorené, sklamanie.",0\n"Nedostatočná efektivita, neodporúčam.",0\n"Menej kvalitný ako vyzerá, sklamanie.",0']


In [93]:
from openai import OpenAI
client = OpenAI()

batch_input_file = client.files.create(
  file=open("batchinput.jsonl", "rb"),
  purpose="batch"
)


In [94]:
batch_input_file_id = batch_input_file.id

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "reviews data "
    }
)

Batch(id='batch_9xbE0JfwCIIXIfXIm5ShZJqP', completion_window='24h', created_at=1714556299, endpoint='/v1/chat/completions', input_file_id='file-cyJSRbGnHx9kL0uwUg9oIsV1', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1714642699, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'reviews data '}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [95]:
client.batches.retrieve("batch_9xbE0JfwCIIXIfXIm5ShZJqP")

Batch(id='batch_9xbE0JfwCIIXIfXIm5ShZJqP', completion_window='24h', created_at=1714556299, endpoint='/v1/chat/completions', input_file_id='file-cyJSRbGnHx9kL0uwUg9oIsV1', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1714557538, error_file_id=None, errors=None, expired_at=None, expires_at=1714642699, failed_at=None, finalizing_at=1714557538, in_progress_at=1714556300, metadata={'description': 'reviews data '}, output_file_id='file-YSIhGaf310zV6RUXt1AnNwsj', request_counts=BatchRequestCounts(completed=4, failed=0, total=4))

In [101]:
content = client.files.content("file-YSIhGaf310zV6RUXt1AnNwsj").content # 10 reviews
# content = client.files.content("batch_qQoYeYrrJQFdjjASb6oRjHUM") # 1000 reviews

In [104]:
with open('data/gpt_reviews.jsonl', 'wb') as file:
    file.write(content)

In [105]:
results = []
with open('data/gpt_reviews.jsonl', 'r') as file:
    for line in file:
        # Parsing the JSON string into a dict and appending to the list of results
        json_object = json.loads(line.strip())
        results.append(json_object)

In [110]:
with open('data/gpt_3.5_reviews.csv', 'rw') as file:
    for res in results[:5]:
        task_id = res['custom_id']
        # Getting index from task id
        index = task_id.split('-')[-1]
        result = res['response']['body']['choices'][0]['message']['content']
        print(result)
        print("\n\n----------------------------\n\n")

review_text,sentiment
Produkt je neuveriteľný, veľmi som spokojný. ,1
Bohužial, kvalita nie je taká ako som očakával.,0
Super výkon a skvelý dizajn. ,1
Veľmi sklamaný, nefunguje podľa očakávaní.,0
Jednoduché ovládanie a vysoká kvalita. ,1
Nevhodný produkt, nespĺňa moje požiadavky. ,0
Vynikajúca hodnota za peniaze, veľmi odporúčam. ,1
Skôr sklamaný, čakal som viac. ,0
Krásny produkt s úžasnými funkciami. ,1
Zlé spracovanie, rýchlo sa poškodzuje. ,0
Presne to, čo som potreboval. ,1
Vôbec nespĺňa očakávania, veľmi sklamaný. ,0
Lahký a praktický, veľká výhoda pre cestovanie. ,1
Veľmi nekvalitný produkt, rýchlo sa pokazil. ,0
Skvelá podpora zákazníkov a rýchle doručenie. ,1
Nesprávna veľkosť, neodpovedá popisu. ,0
Úžasný výkon a dlhá životnosť batérie. ,1
Nepoužiteľný, nefunguje správne. ,0
Moderný a stylový produkt, zákaznícka spokojnosť. ,1
Veľmi vysoká cena, nie je to také dobré. ,0
Dizajn je veľmi pekný a obľúbený. ,1
Funguje len občas, veľmi sklamaný. ,0
Ľahká inštalácia a jednoduché n